In [1]:
using Zygote
using LinearAlgebra
import LinearAlgebra: ldiv!
using SparseArrays
using Arpack
using BenchmarkTools

In [30]:
function f(A,b,dt) 
    # C = cholesky(Matrix(A)) # this works
    D = I + dt * A
    C = cholesky(D) # Sparse array
    x = C \ b
    sum(x)
end
function ldiv!(x::AbstractVector, F::SparseArrays.CHOLMOD.Factor{Float64}, b::Vector{Float64})
    println("ldiv!")
    x .= A \ b
end
function ldiv!(F::SparseArrays.CHOLMOD.Factor{Float64}, b::Vector{Float64})
    println("ldiv!x")
    x = similar(b)
    ldiv!(x, F, b)
end
Zygote.@adjoint function cholesky(::SparseMatrixCSC{Float64, Int64}) 
    Af = cholesky(A)
    function g(y)
        println("adjoint")
        1.0
    end
    Af, g
end
Zygote.@adjoint function SparseArrays.CHOLMOD.Dense{Float64}(x)
    x, 1.0
end


In [31]:
n = 6
A = sprand(n,n,0.2)
A = A * A' + .01 * I
b = rand(n)
t = rand()

# println(f(A,b))
for i=1:300
    grad = Zygote.gradient(f,A,b,t)
    println(grad)
    b -= 0.0001 * grad[3]
    if i % 1 == 0
        println(f(A,b))
    end
end
println(f(A,b))

StackOverflowError: StackOverflowError:

In [29]:
A = sprand(10000,10000, 0.0001)
A = A * A' + .01 * I
function factorize(A)
    D = Matrix(A)
    cholesky(D)
end
@time factorize(A)

  2.321081 seconds (4.81 k allocations: 1.490 GiB, 0.39% gc time, 0.30% compilation time)


Cholesky{Float64, Matrix{Float64}}
U factor:
10000×10000 UpperTriangular{Float64, Matrix{Float64}}:
 0.1  0.0  0.0  0.0  0.0       0.0  …  0.0       0.0       0.0  0.0
  ⋅   0.1  0.0  0.0  0.0       0.0     0.0       0.0       0.0  0.0
  ⋅    ⋅   0.1  0.0  0.0       0.0     0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅   0.1  0.0       0.0     0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅   0.318149  0.0     0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅        0.1  …  0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅         ⋅      0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅         ⋅      0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅         ⋅      0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅         ⋅      0.0       0.0       0.0  0.0
 ⋮                             ⋮    ⋱                           
  ⋅    ⋅    ⋅    ⋅    ⋅         ⋅      0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅    ⋅         ⋅      0.0       0.0       0.0  0.0
  ⋅    ⋅    ⋅    ⋅ 

In [55]:
display(Matrix(I(3)))
display(cholesky(I(3)).L)
display(cholesky(I(3)).U)

function dense_chol(A)
    cholesky(A).L
end
Zygote.jacobian(dense_chol, I(3))[1]

3×3 Matrix{Bool}:
 1  0  0
 0  1  0
 0  0  1

3×3 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅ 
  ⋅    ⋅   1.0

3×3 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅ 
  ⋅    ⋅   1.0

9×9 Matrix{Float64}:
 0.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.5  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.5

In [20]:
N = 10000
A = sprand(N,N,.0005)
A = A * A' + .01 * I
density(A) = count(x->x!=0, A) / (size(A)[1]^2)
println(density(A))
b = rand(N)
function f(A,b)
    D = cholesky(A)
    x = D \ b
    sum(x)
end
# Zygote.gradient(f, A, b)
println("Sparse")
@btime $(A) \ $(b)
println("Sparse Cholesky")
@btime D = cholesky($(A))
@btime $(D) \ $(b)
println("Dense")
@btime D = Matrix($(A))
@btime $(D) \ $(b)
println("Dense Cholesky")
@btime D = cholesky(Matrix($(A)))
@btime $(D) \ $(b);

0.00258926
Sparse


  567.158 ms (66 allocations: 520.06 MiB)
Sparse Cholesky


  552.383 ms (54 allocations: 521.77 MiB)


  59.221 ms (2 allocations: 78.17 KiB)
Dense


  139.791 ms (2 allocations: 762.94 MiB)


  59.739 ms (2 allocations: 78.17 KiB)
Dense Cholesky


  1.718 s (4 allocations: 1.49 GiB)


  60.133 ms (2 allocations: 78.17 KiB)
